# Federal Ethics Compliance Chatbot - Proof of Concept

This notebook demonstrates an agentic RAG system that helps users understand federal ethics violations, penalties, and corrective actions.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv('../.env.local')

# Verify API keys are loaded
required_keys = ['OPENAI_API_KEY', 'TAVILY_API_KEY', 'LANGCHAIN_API_KEY']
for key in required_keys:
    if not os.getenv(key):
        print(f"⚠️ Missing {key} in environment")
    else:
        print(f"✅ {key} configured")

# Set up LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "federal-ethics-chatbot"

✅ OPENAI_API_KEY configured
✅ TAVILY_API_KEY configured
✅ LANGCHAIN_API_KEY configured


In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict
from langchain_core.documents import Document
from typing import List, Optional

class EthicsState(TypedDict):
    question: str
    violation_type: Optional[str]
    context: List[Document]
    web_results: List[dict]
    assessment: str
    response: str

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader

directory_loader = DirectoryLoader("../data", glob="**/*.pdf", loader_cls=PyMuPDFLoader)

ethics_knowledge_resources = directory_loader.load()

print(f"📚 Loaded {len(ethics_knowledge_resources)} pages from federal ethics laws")
print(f"📄 First page preview: {ethics_knowledge_resources[0].page_content[:200]}...")

📚 Loaded 190 pages from federal ethics laws
📄 First page preview: COMPILATION OF 
FEDERAL ETHICS LAWS 
  
 
PREPARED BY THE 
UNITED STATES OFFICE OF GOVERNMENT ETHICS...


In [5]:
ethics_knowledge_resources[0].page_content[:1000]

'COMPILATION OF \nFEDERAL ETHICS LAWS \n  \n \nPREPARED BY THE \nUNITED STATES OFFICE OF GOVERNMENT ETHICS'

In [6]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [7]:
usa_ethics_law_chunks = text_splitter.split_documents(ethics_knowledge_resources)

print(f"🔄 Split {len(ethics_knowledge_resources)} pages into {len(usa_ethics_law_chunks)} chunks")
print(f"📊 Average chunk size: {sum(len(chunk.page_content) for chunk in usa_ethics_law_chunks) // len(usa_ethics_law_chunks)} characters")

🔄 Split 190 pages into 198 chunks
📊 Average chunk size: 2073 characters


In [8]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

embedding_dim = 1536

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

In [10]:
client.create_collection(
    collection_name="ethics_knowledge_index",
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ethics_knowledge_index",
    embedding=embedding_model,
)

In [11]:
vector_store.add_documents(documents=usa_ethics_law_chunks)
print(f"✅ Added {len(usa_ethics_law_chunks)} chunks to ethics knowledge index")

✅ Added 198 chunks to ethics knowledge index


In [12]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [13]:
ethics_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Test retrieval with ethics-specific query
test_results = ethics_retriever.invoke("What are the penalties for accepting gifts from prohibited sources?")
print(f"🔍 Retrieved {len(test_results)} relevant documents")
for i, doc in enumerate(test_results):
    print(f"Document {i+1}: {doc.page_content[:100]}...")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print("---")

🔍 Retrieved 5 relevant documents
Document 1: 155 
 
 
 
18 U.S.C. § 602.  Solicitation of political contributions 
(a) It shall be unlawful for— ...
Source: ../data/Compilation of Federal Ethics Laws (2025).pdf
---
Document 2: 121 
 
 
 
(3) Such listings shall include for each gift of travel or travel expenses— 
(A) the name...
Source: ../data/Compilation of Federal Ethics Laws (2025).pdf
---
Document 3: 114 
 
 
 
IV.  GIFTS AND TRAVEL 
A.  Gifts to Federal employees 
5 U.S.C. § 7353.  Gifts to Federal...
Source: ../data/Compilation of Federal Ethics Laws (2025).pdf
---
Document 4: 116 
 
 
 
B.  Gifts to Superiors 
5 U.S.C. § 7351.  Gifts to superiors 
(a) An employee may not— 
(...
Source: ../data/Compilation of Federal Ethics Laws (2025).pdf
---
Document 5: 156 
 
 
 
18 U.S.C. § 604.  Solicitation from persons on relief 
Whoever solicits or receives or is...
Source: ../data/Compilation of Federal Ethics Laws (2025).pdf
---


In [14]:
def retrieve_ethics_knowledge(state: EthicsState) -> EthicsState:
    """Retrieve relevant federal ethics law documents"""
    retrieved_docs = ethics_retriever.invoke(state["question"])
    return {"context": retrieved_docs}

In [15]:
from langchain_core.prompts import ChatPromptTemplate

ETHICS_ASSESSMENT_TEMPLATE = """
You are a federal ethics compliance expert. Analyze the user's question to determine if it relates to a potential ethics violation.

QUESTION: {question}

FEDERAL ETHICS CONTEXT:
{context}

WEB SEARCH RESULTS:
{web_results}

Provide a comprehensive assessment that includes:
1. **Violation Type**: What type of ethics violation this might be (if any)
2. **Severity Assessment**: Minor, moderate, or serious violation
3. **Legal Penalties**: Specific penalties from federal law
4. **Corrective Actions**: What steps should be taken immediately
5. **Reporting Requirements**: Who should be notified and when
6. **Prevention**: How to avoid similar situations in the future

If no violation is apparent, explain why and provide general guidance.

Base your response primarily on the federal ethics laws provided, supplemented by current web sources for context.
"""

ethics_prompt = ChatPromptTemplate.from_messages([
    ("human", ETHICS_ASSESSMENT_TEMPLATE)
])

In [16]:
# Test the prompt template
test_context = "Federal employees may not accept gifts from prohibited sources..."
test_web = "[{'title': 'Ethics Guidelines', 'content': 'Recent updates...'}]"

test_prompt = ethics_prompt.invoke({
    "question": "Can I accept a lunch invitation from a contractor?",
    "context": test_context,
    "web_results": test_web
})

print("📝 Generated prompt preview:")
print(test_prompt.messages[0].content[:500] + "...")

📝 Generated prompt preview:

You are a federal ethics compliance expert. Analyze the user's question to determine if it relates to a potential ethics violation.

QUESTION: Can I accept a lunch invitation from a contractor?

FEDERAL ETHICS CONTEXT:
Federal employees may not accept gifts from prohibited sources...

WEB SEARCH RESULTS:
[{'title': 'Ethics Guidelines', 'content': 'Recent updates...'}]

Provide a comprehensive assessment that includes:
1. **Violation Type**: What type of ethics violation this might be (if any)
2...


In [17]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o", temperature=0.1)

# Test the model with ethics context
test_response = openai_chat_model.invoke(test_prompt)
print("🤖 Model response preview:")
print(test_response.content[:300] + "...")

🤖 Model response preview:
Based on the federal ethics context provided, let's analyze the question of whether a federal employee can accept a lunch invitation from a contractor.

### 1. Violation Type
The potential ethics violation here relates to the acceptance of gifts from prohibited sources. Federal employees are general...


In [18]:
from langchain_core.output_parsers import StrOutputParser

ethics_chain = ethics_prompt | openai_chat_model | StrOutputParser()

# Test the complete chain
test_result = ethics_chain.invoke({
    "question": "Can I accept a lunch invitation from a contractor?",
    "context": test_context,
    "web_results": test_web
})

print("⚖️ Ethics assessment result:")
print(test_result[:400] + "...")

⚖️ Ethics assessment result:
Based on the federal ethics context provided, let's analyze the question regarding accepting a lunch invitation from a contractor.

### 1. Violation Type:
The potential ethics violation here relates to the acceptance of gifts from prohibited sources. Federal employees are generally prohibited from accepting gifts from contractors or other entities that do business with their agency, as these are c...


In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize web search tool
web_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_domains=["osg.gov", "oge.gov", "ethics.gov", "gsa.gov"]
)

def search_web_sources(state: EthicsState) -> EthicsState:
    """Search for current ethics guidance and recent cases"""
    # Enhance query for better ethics-focused results
    enhanced_query = f"federal ethics violation {state['question']} penalties reporting requirements"

    try:
        web_results = web_search_tool.invoke(enhanced_query)
        return {"web_results": web_results}
    except Exception as e:
        print(f"⚠️ Web search failed: {e}")
        return {"web_results": []}

def assess_ethics_violation(state: EthicsState) -> EthicsState:
    """Generate comprehensive ethics assessment"""

    # Format context for prompt
    context_text = "\\n\\n".join([doc.page_content for doc in state["context"]])
    web_text = str(state.get("web_results", []))

    response = ethics_chain.invoke({
        "question": state["question"],
        "context": context_text,
        "web_results": web_text
    })

    return {"response": response}

/var/folders/yr/gt3hr2h50zv15jd0htzpz2b40000gn/T/ipykernel_19988/1752923716.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


In [20]:
# Build the ethics assessment graph
ethics_graph_builder = StateGraph(EthicsState)

# Add nodes in sequence
ethics_graph_builder.add_node("retrieve", retrieve_ethics_knowledge)
ethics_graph_builder.add_node("search_web", search_web_sources)
ethics_graph_builder.add_node("assess", assess_ethics_violation)

# Define the flow
ethics_graph_builder.add_edge(START, "retrieve")
ethics_graph_builder.add_edge("retrieve", "search_web")
ethics_graph_builder.add_edge("search_web", "assess")

# Compile the graph
ethics_graph = ethics_graph_builder.compile()

print("🔗 Ethics assessment graph compiled successfully")
print("Flow: retrieve → search_web → assess")

🔗 Ethics assessment graph compiled successfully
Flow: retrieve → search_web → assess


In [21]:
from IPython.display import Markdown, display

# Test case 1: Gift acceptance scenario
print("🧪 Test Case 1: Gift Acceptance")
print("=" * 50)

response = ethics_graph.invoke({
    "question": "A contractor offered me tickets to a baseball game worth $150. Can I accept them?"
})

display(Markdown(response["response"]))

🧪 Test Case 1: Gift Acceptance


Based on the information provided and the context of federal ethics laws, let's analyze the situation where a contractor offers tickets to a baseball game worth $150 to a federal employee.

### 1. Violation Type
The potential ethics violation here relates to the acceptance of gifts from outside sources. According to federal ethics regulations, employees are generally prohibited from accepting gifts from prohibited sources or gifts given because of their official position. A contractor doing business with the federal government would typically be considered a prohibited source.

### 2. Severity Assessment
This situation could be considered a **moderate violation**. While the value of the tickets ($150) is not extremely high, accepting gifts from contractors can create a perception of impropriety or a conflict of interest, which is a significant concern in maintaining public trust in government operations.

### 3. Legal Penalties
Under federal ethics laws, specifically the Standards of Ethical Conduct for Employees of the Executive Branch, accepting gifts from prohibited sources can lead to disciplinary actions. While criminal penalties are unlikely for a first-time or minor infraction, administrative actions such as reprimand, suspension, or even termination could be considered depending on the agency's policies and the employee's history.

### 4. Corrective Actions
The employee should **decline the offer** of the tickets to avoid any appearance of impropriety. If the tickets have already been accepted, they should be returned to the contractor with an explanation that accepting such gifts is against federal ethics rules.

### 5. Reporting Requirements
The employee should report the offer to their agency's ethics officer or designated ethics official immediately. This ensures transparency and allows the agency to document the incident and provide guidance on how to handle similar situations in the future.

### 6. Prevention
To avoid similar situations in the future, the employee should:
- Familiarize themselves with the agency's ethics guidelines and the Standards of Ethical Conduct.
- Attend ethics training sessions regularly to stay informed about what constitutes a prohibited gift.
- Consult with the agency's ethics officer whenever in doubt about accepting gifts or other offers.

### General Guidance
Even if no violation is apparent, it is crucial for federal employees to maintain a high standard of ethical conduct. This includes avoiding any actions that could be perceived as compromising their integrity or impartiality. When in doubt, always seek guidance from an ethics official before accepting any gifts or offers from outside sources.

In [22]:
# Test case 2: Outside employment scenario
print("\\n🧪 Test Case 2: Outside Employment")
print("=" * 50)

response = ethics_graph.invoke({
    "question": "I work for the FDA and want to consult for a pharmaceutical company on weekends. Is this allowed?"
})

display(Markdown(response["response"]))

\n🧪 Test Case 2: Outside Employment


Based on the information provided and the federal ethics laws, let's analyze the situation:

1. **Violation Type**: The potential ethics violation in this scenario relates to conflicts of interest and outside employment. Specifically, it involves the possibility of a federal employee engaging in outside employment that could conflict with their official duties or create an appearance of a conflict of interest.

2. **Severity Assessment**: This could be considered a moderate to serious violation, depending on the specific circumstances, such as the nature of the employee's duties at the FDA and the relationship between those duties and the pharmaceutical company.

3. **Legal Penalties**: Federal employees are subject to various penalties for ethics violations, including disciplinary action, fines, or even criminal charges if the conduct violates specific statutes. In this case, if the outside employment creates a conflict of interest or violates post-employment negotiation restrictions, the employee could face disciplinary action or other penalties as determined by the agency's ethics office.

4. **Corrective Actions**: The employee should immediately consult with their agency's ethics office to disclose the potential outside employment and seek guidance. They should refrain from engaging in any outside employment until they receive approval and ensure that it does not conflict with their official duties.

5. **Reporting Requirements**: The employee must report any negotiations or agreements for outside employment to their supervising ethics office within three business days, as required by the STOCK Act. They should also recuse themselves from any official matters that could present a conflict of interest.

6. **Prevention**: To avoid similar situations in the future, the employee should:
   - Regularly review and understand the ethics guidelines and restrictions applicable to their position.
   - Seek advice from the agency's ethics office before engaging in any outside employment or activities that could potentially conflict with their official duties.
   - Ensure full compliance with reporting and recusal requirements to prevent conflicts of interest.

In summary, while the desire to consult for a pharmaceutical company on weekends is not inherently an ethics violation, it raises potential concerns about conflicts of interest and compliance with federal ethics laws. The employee should take proactive steps to ensure that their actions are in line with legal and ethical standards by consulting with their agency's ethics office and adhering to all relevant guidelines and reporting requirements.

In [23]:
# Test case 3: Financial conflict of interest
print("\\n🧪 Test Case 3: Financial Conflict")
print("=" * 50)

response = ethics_graph.invoke({
    "question": "I own stock in a company that my agency is about to investigate. What should I do?"
})

display(Markdown(response["response"]))

\n🧪 Test Case 3: Financial Conflict


Based on the information provided and the context of federal ethics laws, let's analyze the situation:

1. **Violation Type**: The situation described involves a potential conflict of interest under 18 U.S.C. § 208. This statute prohibits federal employees from participating personally and substantially in any particular matter in which they have a financial interest. Owning stock in a company that your agency is about to investigate could create such a conflict.

2. **Severity Assessment**: This could be considered a serious violation if the individual participates in the investigation without addressing the conflict, as it directly affects the integrity of the agency's actions and the public's trust.

3. **Legal Penalties**: Under 18 U.S.C. § 208, violations can lead to criminal penalties, including fines and imprisonment. Additionally, disciplinary actions such as reprimand, suspension, or dismissal may be recommended by the Director of the Office of Government Ethics.

4. **Corrective Actions**: The individual should immediately disclose the financial interest to their agency's ethics official and recuse themselves from any involvement in the investigation. They may also need to divest the stock or establish a blind trust, depending on the guidance from the ethics official.

5. **Reporting Requirements**: The individual must notify their supervising ethics office of the conflict and any recusal actions taken. If the conflict is not resolved, the Director of the Office of Government Ethics may notify the agency head or the President, depending on the individual's position.

6. **Prevention**: To avoid similar situations in the future, the individual should regularly review their financial interests and consult with ethics officials when potential conflicts arise. They should also consider divesting from stocks that could pose conflicts with their official duties.

In summary, the situation presents a potential conflict of interest that must be addressed promptly to comply with federal ethics laws. The individual should take immediate steps to disclose the conflict, recuse themselves, and seek guidance from their agency's ethics official to resolve the issue appropriately.

In [24]:
# Advanced Agentic Ethics Assessment
# This section implements multi-step reasoning for complex ethics scenarios

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END
from typing import Annotated
from langgraph.graph.message import add_messages

class AdvancedEthicsState(TypedDict):
    messages: Annotated[list, add_messages]
    question: str
    context: List[Document]
    web_results: List[dict]
    violation_analysis: str
    severity_assessment: str
    penalty_research: str
    final_guidance: str

# Initialize the reasoning model
reasoning_model = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [25]:
def analyze_violation(state: AdvancedEthicsState) -> AdvancedEthicsState:
    """Step 1: Analyze if there's a violation and what type"""

    system_msg = SystemMessage(content="""
    You are a federal ethics expert specializing in violation identification.
    Analyze the scenario to determine:
    1. Is there a potential violation?
    2. What specific federal ethics law(s) might be violated?
    3. What are the key factors that make this problematic?

    Be specific and cite relevant sections from the federal ethics laws provided.
    """)

    context_text = "\\n\\n".join([doc.page_content for doc in state["context"]])
    human_msg = HumanMessage(content=f"""
    SCENARIO: {state['question']}

    FEDERAL ETHICS LAWS CONTEXT:
    {context_text}

    Provide your violation analysis:
    """)

    response = reasoning_model.invoke([system_msg, human_msg])

    return {
        "messages": [human_msg, response],
        "violation_analysis": response.content
    }

def assess_severity(state: AdvancedEthicsState) -> AdvancedEthicsState:
    """Step 2: Assess the severity level of the violation"""

    system_msg = SystemMessage(content="""
    You are a federal ethics enforcement specialist.
    Based on the violation analysis, determine the severity level:
    - MINOR: Technical violations with minimal impact
    - MODERATE: Clear violations with potential for harm
    - SERIOUS: Violations that undermine public trust or involve significant benefit

    Consider factors like intent, financial value, position of trust, and public impact.
    """)

    human_msg = HumanMessage(content=f"""
    VIOLATION ANALYSIS:
    {state['violation_analysis']}

    Assess the severity level and justify your assessment:
    """)

    response = reasoning_model.invoke([system_msg, human_msg])

    return {
        "messages": state["messages"] + [human_msg, response],
        "severity_assessment": response.content
    }

In [26]:
def research_penalties(state: AdvancedEthicsState) -> AdvancedEthicsState:
    """Step 3: Research specific penalties and consequences"""

    system_msg = SystemMessage(content="""
    You are a federal ethics legal researcher.
    Based on the violation type and severity, research and specify:
    1. Criminal penalties (fines, imprisonment)
    2. Administrative actions (suspension, termination, reprimand)
    3. Civil penalties and restitution requirements
    4. Professional consequences (security clearance, bar discipline)

    Cite specific statutes and precedent cases where available.
    """)

    web_context = "\\n".join([str(result) for result in state.get("web_results", [])])

    human_msg = HumanMessage(content=f"""
    VIOLATION ANALYSIS: {state['violation_analysis']}

    SEVERITY ASSESSMENT: {state['severity_assessment']}

    WEB RESEARCH CONTEXT:
    {web_context}

    Research and specify all applicable penalties:
    """)

    response = reasoning_model.invoke([system_msg, human_msg])

    return {
        "messages": state["messages"] + [human_msg, response],
        "penalty_research": response.content
    }

def generate_final_guidance(state: AdvancedEthicsState) -> AdvancedEthicsState:
    """Step 4: Generate comprehensive guidance and action plan"""

    system_msg = SystemMessage(content="""
    You are a senior federal ethics advisor providing final guidance.
    Synthesize all previous analysis to provide:
    1. Clear summary of the violation and its implications
    2. Immediate actions the person should take
    3. Reporting requirements and deadlines
    4. Steps to minimize harm and demonstrate good faith
    5. Long-term prevention strategies
    6. When to seek legal counsel

    Be practical, actionable, and empathetic while maintaining legal accuracy.
    """)

    human_msg = HumanMessage(content=f"""
    ORIGINAL QUESTION: {state['question']}

    VIOLATION ANALYSIS: {state['violation_analysis']}

    SEVERITY ASSESSMENT: {state['severity_assessment']}

    PENALTY RESEARCH: {state['penalty_research']}

    Provide comprehensive final guidance:
    """)

    response = reasoning_model.invoke([system_msg, human_msg])

    return {
        "messages": state["messages"] + [human_msg, response],
        "final_guidance": response.content
    }

In [27]:
# Build the advanced agentic ethics graph
def setup_advanced_state(question: str, context: List[Document], web_results: List[dict]) -> AdvancedEthicsState:
    """Convert simple inputs to advanced state"""
    return {
        "messages": [],
        "question": question,
        "context": context,
        "web_results": web_results,
        "violation_analysis": "",
        "severity_assessment": "",
        "penalty_research": "",
        "final_guidance": ""
    }

# Create the advanced reasoning graph
advanced_graph_builder = StateGraph(AdvancedEthicsState)

advanced_graph_builder.add_node("analyze", analyze_violation)
advanced_graph_builder.add_node("assess_severity", assess_severity)
advanced_graph_builder.add_node("research_penalties", research_penalties)
advanced_graph_builder.add_node("final_guidance", generate_final_guidance)

# Define the reasoning flow
advanced_graph_builder.add_edge(START, "analyze")
advanced_graph_builder.add_edge("analyze", "assess_severity")
advanced_graph_builder.add_edge("assess_severity", "research_penalties")
advanced_graph_builder.add_edge("research_penalties", "final_guidance")

advanced_ethics_graph = advanced_graph_builder.compile()

print("🧠 Advanced agentic ethics reasoning graph compiled")
print("Flow: analyze → assess_severity → research_penalties → final_guidance")

🧠 Advanced agentic ethics reasoning graph compiled
Flow: analyze → assess_severity → research_penalties → final_guidance


In [28]:
# Complete Ethics Consultation System
# Combines retrieval, web search, and multi-step agentic reasoning

def complete_ethics_consultation(question: str) -> dict:
    """
    Full ethics consultation pipeline:
    1. Retrieve relevant federal ethics law context
    2. Search for current guidance and cases
    3. Apply multi-step agentic reasoning
    4. Return comprehensive assessment
    """

    print(f"🔍 Processing ethics consultation: {question[:60]}...")

    # Step 1: Get initial RAG results
    basic_result = ethics_graph.invoke({"question": question})

    # Step 2: Apply advanced reasoning
    advanced_state = setup_advanced_state(
        question=question,
        context=basic_result["context"],
        web_results=basic_result.get("web_results", [])
    )

    advanced_result = advanced_ethics_graph.invoke(advanced_state)

    return {
        "question": question,
        "basic_assessment": basic_result["response"],
        "detailed_analysis": {
            "violation_analysis": advanced_result["violation_analysis"],
            "severity_assessment": advanced_result["severity_assessment"],
            "penalty_research": advanced_result["penalty_research"],
            "final_guidance": advanced_result["final_guidance"]
        },
        "sources": {
            "federal_law_chunks": len(basic_result["context"]),
            "web_sources": len(basic_result.get("web_results", []))
        }
    }

print("✅ Complete ethics consultation system ready")

✅ Complete ethics consultation system ready


In [29]:
# Comprehensive Test: Complex Ethics Scenario
print("🧪 COMPREHENSIVE TEST: Complex Ethics Scenario")
print("=" * 60)

complex_scenario = """
I am a senior procurement officer at the Department of Defense. Last month,
I participated in a $50M contract award decision that went to TechCorp.
This week, TechCorp offered me a job as their Director of Government Relations
with a $200K salary increase. I haven't accepted yet, but I'm considering it.
Also, my spouse owns 500 shares of TechCorp stock that we bought 2 years ago.
What are the ethics implications and what should I do?
"""

result = complete_ethics_consultation(complex_scenario)

print("\\n📋 BASIC ASSESSMENT:")
print("-" * 30)
display(Markdown(result["basic_assessment"]))

print("\\n🔬 DETAILED VIOLATION ANALYSIS:")
print("-" * 35)
display(Markdown(result["detailed_analysis"]["violation_analysis"]))

print("\\n⚖️ SEVERITY ASSESSMENT:")
print("-" * 25)
display(Markdown(result["detailed_analysis"]["severity_assessment"]))

🧪 COMPREHENSIVE TEST: Complex Ethics Scenario
🔍 Processing ethics consultation: 
I am a senior procurement officer at the Department of Defe...
\n📋 BASIC ASSESSMENT:
------------------------------


Based on the information provided and the relevant federal ethics laws, let's analyze the situation:

### 1. Violation Type
The situation potentially involves several types of ethics violations:

- **Post-Employment Negotiation Restrictions**: Under the STOCK Act, federal employees must report any employment negotiations with a private entity within three business days. The user has not indicated that they have reported this job offer to their supervising ethics office, which could be a violation.
  
- **Conflict of Interest**: The user's spouse owns stock in TechCorp, which could create a conflict of interest or the appearance of one, especially since the user was involved in awarding a contract to TechCorp.

- **Procurement Integrity**: As a procurement officer, the user must avoid any actions that could be perceived as using their position for personal gain, which includes negotiating employment with a contractor they recently awarded a contract to.

### 2. Severity Assessment
- **Serious Violation**: The potential violations are serious due to the high value of the contract involved ($50M) and the significant salary increase offered, which could be perceived as a quid pro quo arrangement.

### 3. Legal Penalties
- **STOCK Act Violations**: Failure to report employment negotiations can lead to administrative penalties, including disciplinary action.
  
- **Conflict of Interest**: Violations can result in fines, imprisonment, or both, depending on the severity and intent.

- **Procurement Integrity Violations**: These can lead to criminal charges, fines, and imprisonment, especially if there is evidence of using insider information or influence for personal gain.

### 4. Corrective Actions
- **Immediate Reporting**: The user should immediately report the job offer to their supervising ethics office and the designated agency ethics official.
  
- **Recusal**: The user should recuse themselves from any further involvement in matters related to TechCorp to avoid any appearance of a conflict of interest.

- **Divestiture**: Consider divesting the TechCorp stock owned by the spouse to eliminate any potential conflict of interest.

### 5. Reporting Requirements
- **Supervising Ethics Office**: The user must report the job offer and any negotiations to their supervising ethics office within three business days of the offer.
  
- **Agency Ethics Official**: Notify the designated agency ethics official about the potential conflict of interest and any steps taken to mitigate it.

### 6. Prevention
- **Training**: Regular ethics training to understand post-employment restrictions and conflict of interest laws.
  
- **Consultation**: Regularly consult with the agency's ethics office when in doubt about potential conflicts or post-employment opportunities.

- **Documentation**: Keep thorough records of all communications and decisions related to procurement and employment negotiations to ensure transparency and accountability.

In summary, the user is potentially facing serious ethics violations related to post-employment negotiations and conflicts of interest. Immediate corrective actions and reporting are necessary to address these issues and prevent further complications.

\n🔬 DETAILED VIOLATION ANALYSIS:
-----------------------------------


1. **Potential Violation:**
   Yes, there is a potential violation in this scenario.

2. **Specific Federal Ethics Law(s) Potentially Violated:**
   - **41 U.S.C. § 2103**: This statute requires an agency official who is participating personally and substantially in a federal agency procurement to report any contact regarding possible non-Federal employment with a bidder or offeror in that procurement. The official must either reject the possibility of employment or disqualify themselves from further participation in the procurement process.
   - **Pub. L. 112-105 § 17 (STOCK Act)**: This law requires individuals who are negotiating for future employment to file a statement with their supervising ethics office within three business days of the commencement of such negotiations.

3. **Key Factors Making This Problematic:**
   - **Conflict of Interest**: As a senior procurement officer who participated in awarding a contract to TechCorp, considering a job offer from them creates a conflict of interest or at least the appearance of one. This is particularly concerning given the significant salary increase offered, which could be perceived as a reward for the contract award.
   - **Failure to Report Employment Negotiations**: The officer has not yet reported the job offer from TechCorp to their supervising ethics office, as required by the STOCK Act. This lack of disclosure is a violation of the post-employment negotiation restrictions.
   - **Spouse's Financial Interest**: The ownership of TechCorp stock by the officer's spouse could further complicate the conflict of interest, as it may appear that the officer's decisions could benefit their personal financial interests.

**Recommended Actions:**
- **Immediate Disclosure**: The officer should immediately report the job offer from TechCorp to their supervisor and the designated agency ethics official, as required by 41 U.S.C. § 2103 and the STOCK Act.
- **Recusal**: The officer should recuse themselves from any further involvement in matters related to TechCorp to avoid any actual or perceived conflicts of interest.
- **Consult Ethics Office**: The officer should consult with their agency's ethics office to discuss the implications of their spouse's stock ownership and consider whether divestiture is necessary to comply with conflict-of-interest laws.

\n⚖️ SEVERITY ASSESSMENT:
-------------------------


Based on the violation analysis provided, the severity level of this situation is **SERIOUS**. Here's the justification for this assessment:

1. **Intent and Position of Trust**: The senior procurement officer holds a position of significant trust and responsibility, directly involved in federal procurement processes. The intent to negotiate employment with a company that was awarded a contract under their purview suggests a potential misuse of their position for personal gain.

2. **Financial Value and Benefit**: The job offer from TechCorp includes a significant salary increase, which could be perceived as a reward for the contract award. Additionally, the officer's spouse owns stock in TechCorp, which could lead to personal financial gain from decisions made by the officer.

3. **Public Impact and Trust**: The failure to report employment negotiations and the potential conflict of interest undermine public trust in the integrity of the federal procurement process. The appearance of impropriety, especially in a high-stakes procurement environment, can damage the reputation of the agency and erode confidence in government operations.

4. **Legal Violations**: The officer's actions violate specific federal ethics laws, including the requirement to report employment negotiations and the need to avoid conflicts of interest. These violations are not merely technical but have substantial implications for ethical governance.

Given these factors, the situation involves significant ethical breaches that could undermine public trust and involve considerable personal benefit, warranting a classification of SERIOUS.

In [30]:
print("\\n💰 PENALTY RESEARCH:")
print("-" * 20)
display(Markdown(result["detailed_analysis"]["penalty_research"]))

print("\\n🎯 FINAL GUIDANCE:")
print("-" * 18)
display(Markdown(result["detailed_analysis"]["final_guidance"]))

print(f"\\n📊 SOURCES CONSULTED:")
print(f"- Federal law chunks: {result['sources']['federal_law_chunks']}")
print(f"- Web sources: {result['sources']['web_sources']}")

print("\\n✅ Test completed successfully!")

\n💰 PENALTY RESEARCH:
--------------------


Based on the potential violations and the severity assessment provided, here are the applicable penalties and consequences for the senior procurement officer's actions:

1. **Criminal Penalties:**
   - **41 U.S.C. § 2105**: Violations of procurement integrity laws, such as those under 41 U.S.C. § 2103, can result in criminal penalties. If convicted, the officer could face fines and imprisonment. Specifically, under 18 U.S.C. § 216, criminal penalties for violating conflict of interest statutes can include fines up to $50,000 and imprisonment for up to five years.
   - **18 U.S.C. § 208**: If the officer's actions are found to involve a conflict of interest due to their spouse's financial interest, they could face additional criminal penalties under this statute, which prohibits federal employees from participating in matters where they have a financial interest.

2. **Administrative Actions:**
   - **Suspension or Termination**: Given the seriousness of the violation, the officer could face suspension or termination from their position. Agencies have the authority to take such actions to maintain the integrity of their operations.
   - **Reprimand**: At a minimum, the officer could receive a formal reprimand, which would be documented in their personnel file.

3. **Civil Penalties and Restitution Requirements:**
   - **Civil Fines**: Under 41 U.S.C. § 2105, civil penalties for violations of procurement integrity laws can include fines up to $50,000 for each violation or the amount of compensation received or offered for the prohibited conduct, whichever is greater.
   - **Restitution**: If the officer's actions resulted in financial gain, they might be required to make restitution to the government or other affected parties.

4. **Professional Consequences:**
   - **Security Clearance**: The officer's security clearance could be revoked or suspended due to the breach of trust and potential conflict of interest, affecting their ability to work in positions requiring such clearance.
   - **Bar Discipline**: If the officer is a licensed attorney, they could face disciplinary actions from their state bar association for ethical violations, which could include suspension or disbarment.

**Precedent Cases:**
- **United States v. Sawyer, 85 F.3d 713 (1st Cir. 1996)**: This case involved a public official who engaged in negotiations for future employment with a company while participating in matters affecting that company, highlighting the importance of disclosing such negotiations to avoid conflicts of interest.
- **United States v. Smith, 267 F.3d 1154 (D.C. Cir. 2001)**: This case addressed the issue of conflicts of interest and the importance of recusal to maintain the integrity of federal procurement processes.

These penalties and consequences underscore the importance of adhering to federal ethics laws and maintaining transparency in employment negotiations to avoid conflicts of interest and preserve public trust in government operations.

\n🎯 FINAL GUIDANCE:
------------------


1. **Clear Summary of the Violation and Its Implications:**
   You are facing a serious ethical violation due to your involvement in a $50M contract award to TechCorp and the subsequent job offer from them. This situation presents a conflict of interest, as it could appear that the job offer is a reward for the contract decision. Additionally, your spouse's ownership of TechCorp stock further complicates the conflict of interest. Failure to report the job offer and recuse yourself from related matters violates federal ethics laws, undermining public trust and potentially leading to severe legal and professional consequences.

2. **Immediate Actions You Should Take:**
   - **Report the Job Offer:** Immediately disclose the job offer from TechCorp to your supervisor and the designated agency ethics official. This is required under the STOCK Act and 41 U.S.C. § 2103.
   - **Recusal:** Cease any involvement in matters related to TechCorp to prevent further conflicts of interest.
   - **Consult Ethics Office:** Engage with your agency's ethics office to discuss the implications of your spouse's stock ownership and determine if divestiture is necessary.

3. **Reporting Requirements and Deadlines:**
   - **Employment Negotiations Disclosure:** File a statement with your supervising ethics office within three business days of receiving the job offer, as mandated by the STOCK Act.
   - **Conflict of Interest Disclosure:** Report any potential conflicts related to your spouse's stock ownership to your ethics office immediately.

4. **Steps to Minimize Harm and Demonstrate Good Faith:**
   - **Voluntary Disclosure:** Proactively disclose all relevant information to your ethics office and cooperate fully with any investigations.
   - **Divestiture Consideration:** Consider divesting your spouse's TechCorp stock to eliminate any appearance of conflict.
   - **Public Statement:** If appropriate, issue a statement acknowledging the situation and your commitment to resolving it ethically.

5. **Long-term Prevention Strategies:**
   - **Ethics Training:** Participate in regular ethics training to stay informed about federal ethics laws and avoid future violations.
   - **Establish Clear Boundaries:** Develop clear personal and professional boundaries to prevent conflicts of interest.
   - **Regular Ethics Consultations:** Schedule periodic consultations with your ethics office to review potential conflicts and ensure compliance.

6. **When to Seek Legal Counsel:**
   - **Immediate Legal Advice:** Given the seriousness of the situation, seek legal counsel immediately to understand your rights and obligations.
   - **If Facing Disciplinary Action:** If you face suspension, termination, or other disciplinary actions, legal representation will be crucial.
   - **Potential Criminal Charges:** If there is a possibility of criminal charges, legal counsel will be essential to navigate the legal process and protect your interests.

By taking these steps, you can address the current ethical concerns, demonstrate your commitment to integrity, and prevent future violations.

\n📊 SOURCES CONSULTED:
- Federal law chunks: 5
- Web sources: 81
\n✅ Test completed successfully!


In [31]:
# Interactive Ethics Consultation Interface
def ethics_chatbot():
    """Simple interactive interface for ethics consultations"""

    print("🏛️ Federal Ethics Compliance Chatbot")
    print("=" * 40)
    print("Ask about potential ethics violations, penalties, and guidance.")
    print("Type 'exit' to quit.\\n")

    while True:
        question = input("Your ethics question: ").strip()

        if question.lower() in ['exit', 'quit', 'bye']:
            print("👋 Thank you for using the ethics consultation system!")
            break

        if not question:
            continue

        try:
            result = complete_ethics_consultation(question)
            print("\\n" + "="*50)
            print("ETHICS CONSULTATION RESPONSE")
            print("="*50)
            display(Markdown(result["detailed_analysis"]["final_guidance"]))
            print("\\n" + "-"*50 + "\\n")

        except Exception as e:
            print(f"❌ Error processing question: {e}")
            print("Please try rephrasing your question.\\n")

print("🤖 Interactive chatbot ready. Call ethics_chatbot() to start.")

🤖 Interactive chatbot ready. Call ethics_chatbot() to start.


In [32]:
# Summary: Ethics Chatbot Architecture

print("📝 FEDERAL ETHICS CHATBOT - SYSTEM SUMMARY")
print("=" * 50)

print("""
🏗️ ARCHITECTURE:
├── Data Layer: Federal Ethics Laws PDF (190 pages)
├── Vector Store: Qdrant in-memory with OpenAI embeddings
├── Web Search: Tavily for current guidance and cases
└── Agentic Reasoning: Multi-step LLM analysis

🤖 AGENTIC WORKFLOW:
1. Question Analysis & Knowledge Retrieval
2. Web Search for Current Context
3. Violation Analysis (identify laws and factors)
4. Severity Assessment (minor/moderate/serious)
5. Penalty Research (criminal, civil, administrative)
6. Final Guidance (actions, reporting, prevention)

✅ CAPABILITIES:
- Identifies potential federal ethics violations
- Assesses severity and legal implications
- Provides specific penalty information
- Offers actionable corrective guidance
- Cites relevant federal laws and regulations
- Incorporates current web-based guidance

🎯 USE CASES:
- Gift acceptance scenarios
- Outside employment conflicts
- Financial conflicts of interest
- Post-employment restrictions
- Procurement ethics violations

📊 EVALUATION READY:
- Multi-source retrieval (federal law + web)
- Structured reasoning pipeline
- Comprehensive response format
- Citation and source tracking
""")

📝 FEDERAL ETHICS CHATBOT - SYSTEM SUMMARY

🏗️ ARCHITECTURE:
├── Data Layer: Federal Ethics Laws PDF (190 pages)
├── Vector Store: Qdrant in-memory with OpenAI embeddings
├── Web Search: Tavily for current guidance and cases
└── Agentic Reasoning: Multi-step LLM analysis

🤖 AGENTIC WORKFLOW:
1. Question Analysis & Knowledge Retrieval
2. Web Search for Current Context
3. Violation Analysis (identify laws and factors)
4. Severity Assessment (minor/moderate/serious)
5. Penalty Research (criminal, civil, administrative)
6. Final Guidance (actions, reporting, prevention)

✅ CAPABILITIES:
- Identifies potential federal ethics violations
- Assesses severity and legal implications
- Provides specific penalty information
- Offers actionable corrective guidance
- Cites relevant federal laws and regulations
- Incorporates current web-based guidance

🎯 USE CASES:
- Gift acceptance scenarios
- Outside employment conflicts
- Financial conflicts of interest
- Post-employment restrictions
- Procurement

In [33]:
# Next Steps for Full Application Development
print("🚀 NEXT DEVELOPMENT PHASES")
print("=" * 30)

next_steps = """
PHASE 1 - DOCKER DEPLOYMENT:
□ Create FastAPI backend service
□ Build React frontend interface
□ Set up Docker Compose stack
□ Add environment configuration

PHASE 2 - PRODUCTION FEATURES:
□ User authentication and sessions
□ Conversation history storage
□ Document upload for case-specific analysis
□ Advanced retrieval (hybrid search, reranking)

PHASE 3 - EVALUATION & MONITORING:
□ RAGAS evaluation framework setup
□ Synthetic test dataset generation
□ Performance metrics dashboard
□ LangSmith observability integration

PHASE 4 - ADVANCED RETRIEVAL:
□ Implement GraphRAG for legal relationships
□ Add semantic reranking with cross-encoders
□ Multi-hop reasoning for complex scenarios
□ Citation accuracy verification
"""

print(next_steps)
print("\\n📋 Current POC Status: ✅ COMPLETE")
print("Ready for Task 5 (RAGAS evaluation) and full stack development!")

🚀 NEXT DEVELOPMENT PHASES

PHASE 1 - DOCKER DEPLOYMENT:
□ Create FastAPI backend service
□ Build React frontend interface
□ Set up Docker Compose stack
□ Add environment configuration

PHASE 2 - PRODUCTION FEATURES:
□ User authentication and sessions
□ Conversation history storage
□ Document upload for case-specific analysis
□ Advanced retrieval (hybrid search, reranking)

PHASE 3 - EVALUATION & MONITORING:
□ RAGAS evaluation framework setup
□ Synthetic test dataset generation
□ Performance metrics dashboard
□ LangSmith observability integration

PHASE 4 - ADVANCED RETRIEVAL:
□ Implement GraphRAG for legal relationships
□ Add semantic reranking with cross-encoders
□ Multi-hop reasoning for complex scenarios
□ Citation accuracy verification

\n📋 Current POC Status: ✅ COMPLETE
Ready for Task 5 (RAGAS evaluation) and full stack development!


In [ ]:
# Run this cell to test the complete system
# ethics_chatbot()  # Uncomment to start interactive session